In [1]:
import matplotlib.colors as mcolors
import matplotlib.animation as ma
import matplotlib.pyplot as plt
from itertools import product
import numpy as np
import numba as nb
import imageio
import shutil
import sys
import os

if "ipykernel_launcher.py" in sys.argv[0]:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

randomSeed = 100

%matplotlib inline
%config InlineBackend.figure_format = "retina"

new_cmap = mcolors.LinearSegmentedColormap.from_list(
    "new", plt.cm.jet(np.linspace(0, 1, 256)) * 0.85, N=256
)

@nb.njit
def colors_idx(phaseTheta):
    return np.floor(256 - phaseTheta / (2 * np.pi) * 256).astype(np.int32)

import seaborn as sns

sns.set_theme(
    style="ticks", 
    font_scale=1.1, rc={
    'figure.figsize': (6, 5),
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'grid.color': '#dddddd',
    'grid.linewidth': 0.5,
    "lines.linewidth": 1.5,
    'text.color': '#000000',
    'figure.titleweight': "bold",
    'xtick.color': '#000000',
    'ytick.color': '#000000'
})

plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'

from main import *
from multiprocessing import Pool
import pandas as pd

colors = ["#403990", "#80A6E2", "#FBDD85", "#F46F43", "#CF3D3E"]
cmap = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors)
cmap_r = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors[::-1])

In [5]:
import matplotlib.patheffects as pe

colors = ["#403990", "#80A6E2", "#FBDD85", "#F46F43", "#CF3D3E"]
cmap = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors)
class1, class2 = (
    np.concatenate([np.ones(500), np.zeros(500)]).astype(bool), 
    np.concatenate([np.zeros(500), np.ones(500)]).astype(bool)
)
alphaRate = 0.9
scale = 35
width = 0.003

def draw_mp4(sa: StateAnalysis, fileName: str, mp4Path: str = "./normalMP4", step: int = 1, tailCounts: int = None):
    
    def plot_frame(i):
        pbar.update(1)
        fig.clear()
        ax = plt.subplot(1, 1, 1)
        singlePositionX, singlePhaseTheta, _ = sa.get_state(index=i)
        omegaTheta = sa.model.omegaTheta
        if isinstance(sa.model, SingleDistribution):
            ax.quiver(
                singlePositionX[:, 0], singlePositionX[:, 1],
                np.cos(singlePhaseTheta[:]), np.sin(singlePhaseTheta[:]), 
                color=[cmap(i) for i in (omegaTheta - 1) / 2], alpha=1,
                scale=23, width=0.005
            )
        elif isinstance(sa.model, SpatialGroups):
            ax.quiver(
                singlePositionX[class1, 0], singlePositionX[class1, 1],
                np.cos(singlePhaseTheta[class1]), np.sin(singlePhaseTheta[class1]), color='red', 
                # alpha = (1 - alphaRate) + (np.abs(omegaTheta[class1]) - 1) / 2 * alphaRate,
                scale=scale, width=width
            )
            ax.quiver(
                singlePositionX[class2, 0], singlePositionX[class2, 1],
                np.cos(singlePhaseTheta[class2]), np.sin(singlePhaseTheta[class2]), color='#414CC7', 
                # alpha = (1 - alphaRate) + (np.abs(omegaTheta[class2]) - 1) / 2 * alphaRate,
                scale=scale, width=width
            )
        strengthLambda = np.round(sa.model.strengthLambda, 3)
        distanceD0 = np.round(sa.model.distanceD0, 2)
        ax.set_title(f"$\lambda={strengthLambda},\ d_0={distanceD0}$", fontweight="bold")
        ax.set_xlim(0, 10)
        ax.set_ylim(0, 10)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.tight_layout()

    fig, _ = plt.subplots(1, 1, figsize=(4, 4))
    TNum = sa.TNum
    startIdx = TNum - tailCounts * step if tailCounts else 0
    frames = np.arange(startIdx, TNum, step)
    pbar = tqdm(total=len(frames))
    ani = ma.FuncAnimation(fig, plot_frame, frames=frames, interval=40, repeat=False)
    ani.save(f"{mp4Path}/{fileName}.mp4", dpi=200)
    plt.close()
    pbar.close()

def draw_mp4s(sas: List[StateAnalysis], fileName: str, mp4Path: str = "./normalMP4", step: int = 1, tailCounts: int = None):
    
    def plot_frame(i):
        pbar.update(1)
        fig.clear()
        for idx in range(len(sas)):
            ax = plt.subplot(rowNum, 3, idx + 1)
            sa = sas[idx]
            singlePositionX, singlePhaseTheta, _ = sa.get_state(index=i)
            omegaTheta = sa.model.omegaTheta
            if isinstance(sa.model, SingleDistribution):
                ax.quiver(
                    singlePositionX[:, 0], singlePositionX[:, 1],
                    np.cos(singlePhaseTheta[:]), np.sin(singlePhaseTheta[:]), 
                    color=[cmap(i) for i in (omegaTheta - 1) / 2], alpha=1,
                    scale=23, width=0.005
                )
            elif isinstance(sa.model, SpatialGroups):
                ax.quiver(
                    singlePositionX[class1, 0], singlePositionX[class1, 1],
                    np.cos(singlePhaseTheta[class1]), np.sin(singlePhaseTheta[class1]), color='red', 
                    # alpha = (1 - alphaRate) + (np.abs(omegaTheta[class1]) - 1) / 2 * alphaRate,
                    scale=scale, width=width
                )
                ax.quiver(
                    singlePositionX[class2, 0], singlePositionX[class2, 1],
                    np.cos(singlePhaseTheta[class2]), np.sin(singlePhaseTheta[class2]), color='#414CC7', 
                    # alpha = (1 - alphaRate) + (np.abs(omegaTheta[class2]) - 1) / 2 * alphaRate,
                    scale=scale, width=width
                )
            strengthLambda = np.round(sa.model.strengthLambda, 3)
            distanceD0 = np.round(sa.model.distanceD0, 2)
            ax.set_title(f"({chr(65 + idx).lower()})  $\lambda={strengthLambda},\ d_0={distanceD0}$", fontweight="bold")
            ax.set_xlim(0, 10)
            ax.set_ylim(0, 10)
            ax.set_xticks([])
            ax.set_yticks([])
        plt.tight_layout()

    rowNum = len(sas) // 3
    fig, axs = plt.subplots(rowNum, 3, figsize=(10, rowNum * 4))
    axs = axs.flatten()
    TNum = sas[0].TNum
    startIdx = TNum - tailCounts * step if tailCounts else 0
    frames = np.arange(startIdx, TNum, step)
    pbar = tqdm(total=len(frames))
    ani = ma.FuncAnimation(fig, plot_frame, frames=frames, interval=40, repeat=False)
    ani.save(f"{mp4Path}/{fileName}.mp4", dpi=200)
    plt.close()
    pbar.close()

def draw_time_vary(sa: StateAnalysis, fileName: str = None, mp4Path: str = "./normalMP4", step: int = 1, earlyStop: int = None):
    if fileName is None:
        fileName = f"tv_{sa.model}"

    def plot_frame(i):
        pbar.update(1)
        frameIdx = np.where(frames == i)[0][0]

        fig.clear()
        ax1 = plt.subplot2grid((3, 2), (0, 0), rowspan=2)
        ax2 = plt.subplot2grid((3, 2), (0, 1), rowspan=2)
        ax3 = plt.subplot2grid((3, 2), (2, 0), colspan=2)
        singlePositionX, singlePhaseTheta, _ = sa.get_state(index=i)
        omegaTheta = sa.model.omegaTheta

        # plot the snapshot
        if isinstance(sa.model, SingleDistribution):
            ax1.quiver(
                singlePositionX[:, 0], singlePositionX[:, 1],
                np.cos(singlePhaseTheta[:]), np.sin(singlePhaseTheta[:]), 
                color=[cmap(i) for i in (omegaTheta - 1) / 2], alpha=1,
                scale=23, width=0.005
            )
        elif isinstance(sa.model, SpatialGroups):
            ax1.quiver(
                singlePositionX[class1, 0], singlePositionX[class1, 1],
                np.cos(singlePhaseTheta[class1]), np.sin(singlePhaseTheta[class1]), color='red', 
                scale=scale, width=width
            )
            ax1.quiver(
                singlePositionX[class2, 0], singlePositionX[class2, 1],
                np.cos(singlePhaseTheta[class2]), np.sin(singlePhaseTheta[class2]), color='#414CC7', 
                scale=scale, width=width
            )

        for classNum in classes:
            classIdxs = classes[classNum]
            meanPosition = centersPool[frameIdx][classIdxs[0]]
            text = ax1.text(meanPosition[0], meanPosition[1], rf"$C_{classNum}$", fontsize=16, color="black")
            text.set_path_effects([pe.withStroke(linewidth=1, foreground="white")])
        
        strengthLambda = np.round(sa.model.strengthLambda, 3)
        distanceD0 = np.round(sa.model.distanceD0, 2)
        ax1.set_title(f"$\lambda={strengthLambda},\ d_0={distanceD0}$", fontweight="bold")
        ax1.set_xlim(0, 10)
        ax1.set_ylim(0, 10)
        ax1.set_xlabel(r"$x$", fontsize=16)
        ax1.set_ylabel(r"$y$", fontsize=16, rotation=0)
        ax1.set_xticks([])
        ax1.set_yticks([])

        # plot the adjacency matrix
        soredPosition = singlePositionX[idxs]
        deltaX = SpatialGroups._delta_x(soredPosition, soredPosition[:, np.newaxis], 10, 5)
        distances = SpatialGroups.distance_x(deltaX)
        Aij = distances <= sa.model.distanceD0
        for idx, row in enumerate(Aij):
            isTrue = np.where(row)[0]
            ax2.scatter(np.ones_like(isTrue) * idx, isTrue, color=colors[idx], s=2, alpha=0.6)

        for classNum in classes:
            classIdxs = classes[classNum]
            AijPosition = np.sum([len(classes[cNum]) for cNum in classes if cNum < classNum]) + len(classIdxs) / 2
            text = ax2.text(AijPosition, AijPosition, rf"$C_{classNum}$", fontsize=18, color="black", fontweight="bold")
            text.set_path_effects([pe.withStroke(linewidth=1, foreground="white")])

        ax2.tick_params(direction='in', length=3)
        ax2.spines['right'].set_color('none')
        ax2.spines['top'].set_color('none')
        ax2.set_xlabel(r"$i$", fontsize=16, labelpad=-10)
        ax2.set_ylabel(r"$j$", fontsize=16, rotation=0, labelpad=-10)
        ax2.grid(True)
        ax2.set_xlim(-0.5, len(idxs) + 0.5)
        ax2.set_ylim(-0.5, len(idxs) + 0.5)

        # plot the order parameter
        ax3.plot(frames * sa.model.dt, opR, label=r"$R$ (left)")
        ax3.plot(frames * sa.model.dt, opRc, label=r"$R_c$ (left)")
        ax3.scatter(i * sa.model.dt, opR[frameIdx], facecolor="white", edgecolors="C0", s=40, linewidths=2, zorder=10)
        ax3.scatter(i * sa.model.dt, opRc[frameIdx], facecolor="white", edgecolors="C1", s=40, linewidths=2, zorder=10)
        ax3.set_ylim(-0.35, 1 + 0.05)
        ax3.set_ylabel("Order Parameter", fontsize=16)
        ax3.legend(loc="lower left", ncol=2, frameon=False)
        ax3.tick_params(direction='in', length=3)
        ax3.set_xlabel(r"$t$", fontsize=16)
        ax3 = ax3.twinx()
        ax3.plot(frames * sa.model.dt, opDeltaOmega, color="C2", label=r"$\Delta \Omega$ (right)")
        ax3.scatter(i * sa.model.dt, opDeltaOmega[frameIdx], facecolor="white", edgecolors="C2", s=40, linewidths=2, zorder=10)
        ax3.set_ylim(-np.max(opDeltaOmega) * 0.25, np.max(opDeltaOmega) * 1.1)
        ax3.legend(loc="lower right", ncol=1, frameon=False)
        ax3.grid(True)
        ax3.tick_params(direction='in', length=3)

        # plt.tight_layout()
        # assert 0

    lastPositionX, lastPhaseTheta, lastPointTheta = sa.get_state(index=-1)
    centers = StateAnalysis._calc_centers(
    lastPositionX, lastPhaseTheta, lastPointTheta, 3, 0.01
    )
    classes = StateAnalysis._calc_classes(
        centers, 0.3, StateAnalysis._adj_distance(centers, centers[:, np.newaxis], 10, 5)
    )
    classes = {i + 1: classes[i] for i in range(len(classes))}
    clusterColors = [cmap(i / len(classes)) for i in range(len(classes))]
    colors = np.concatenate([[clusterColors[i - 1]] * len(classes[i]) for i in range(1, len(classes) + 1)])
    idxs = np.concatenate(list(classes.values()))

    fig, _ = plt.subplots(1, 1, figsize=(11, 8))
    # fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)

    if earlyStop is None:
        TNum = sa.TNum - 1000
    else:
        TNum = min(earlyStop, sa.TNum - 1000)
    startIdx = 1
    frames = np.arange(startIdx, TNum, step)

    # calc order parameters
    selectedPositionX = sa.totalPositionX[frames]
    selectedPhaseTheta = sa.totalPhaseTheta[frames]
    selectedPointTheta = sa.totalPointTheta[frames]
    opR = np.abs(np.mean(np.cos(selectedPhaseTheta) + 1j * np.sin(selectedPhaseTheta), axis=1))
    opRc = np.zeros_like(opR)
    opDeltaOmega = np.zeros_like(opR)
    centersPool = list()
    for idx in tqdm(range(len(frames)), desc="Calculating order parameters"):
        lastPositionX = selectedPositionX[idx]
        lastPhaseTheta = selectedPhaseTheta[idx]
        lastPointTheta = selectedPointTheta[idx]
        centers = StateAnalysis._calc_centers(lastPositionX, lastPhaseTheta, lastPointTheta, 
                                              sa.model.speedV, sa.model.dt)
        modCenters = np.mod(centers, 10)
        centersPool.append(modCenters)
        tvClasses = StateAnalysis._calc_classes(
            modCenters, 0.3, 
            StateAnalysis._adj_distance(modCenters, modCenters[:, np.newaxis], 10, 5)
        )
        counts = 0
        sumRc = 0
        sumDeltaOmega = 0
        for classOcsis in tvClasses:
            if len(classOcsis) <= 5:
                continue
            sumRc += StateAnalysis._clac_phase_sync_op(lastPhaseTheta[classOcsis])
            counts += 1

            meanPointTheta = selectedPointTheta[idx: idx + 1000, classOcsis].mean(axis=0) / sa.model.dt
            sumDeltaOmega += ((meanPointTheta - meanPointTheta[:, np.newaxis])**2).sum() / len(classOcsis) ** 2

        opRc[idx] = sumRc / counts
        opDeltaOmega[idx] = sumDeltaOmega / counts

    # draw
    pbar = tqdm(total=len(frames) + 1)
    ani = ma.FuncAnimation(fig, plot_frame, frames=frames, interval=20 * step, repeat=False)
    ani.save(f"{mp4Path}/{fileName}.mp4", dpi=200)
    plt.close()
    pbar.close()

def draw_mp4s_row2_Aij(sas: List[StateAnalysis], fileName: str, mp4Path: str = "./normalMP4", 
                       step: int = 1, tailCounts: int = None, earlyStop: int = None):
    
    def plot_frame(i):
        pbar.update(1)
        fig.clear()
        for saIdx in range(len(sas)):
            ax1 = plt.subplot(2, 3, saIdx + 1)
            sa = sas[saIdx]
            singlePositionX, singlePhaseTheta, _ = sa.get_state(index=i)
            omegaTheta = sa.model.omegaTheta
            if isinstance(sa.model, SingleDistribution):
                ax1.quiver(
                    singlePositionX[:, 0], singlePositionX[:, 1],
                    np.cos(singlePhaseTheta[:]), np.sin(singlePhaseTheta[:]), 
                    color=[cmap(i) for i in (omegaTheta - 1) / 2], alpha=1,
                    scale=23, width=0.005
                )
            elif isinstance(sa.model, SpatialGroups):
                ax1.quiver(
                    singlePositionX[class1, 0], singlePositionX[class1, 1],
                    np.cos(singlePhaseTheta[class1]), np.sin(singlePhaseTheta[class1]), color='red', 
                    # alpha = (1 - alphaRate) + (np.abs(omegaTheta[class1]) - 1) / 2 * alphaRate,
                    scale=scale, width=width
                )
                ax1.quiver(
                    singlePositionX[class2, 0], singlePositionX[class2, 1],
                    np.cos(singlePhaseTheta[class2]), np.sin(singlePhaseTheta[class2]), color='#414CC7', 
                    # alpha = (1 - alphaRate) + (np.abs(omegaTheta[class2]) - 1) / 2 * alphaRate,
                    scale=scale, width=width
                )

            # classes = totalClasses[saIdx]
            
            strengthLambda = np.round(sa.model.strengthLambda, 3)
            distanceD0 = np.round(sa.model.distanceD0, 2)
            ax1.set_title(f"({chr(65 + saIdx).lower()})  $\lambda={strengthLambda},\ d_0={distanceD0}$", fontweight="bold")
            ax1.set_xlim(0, 10)
            ax1.set_ylim(0, 10)
            ax1.set_xticks([])
            ax1.set_yticks([])

            ax2 = plt.subplot(2, 3, saIdx + 4)
            soredPosition = singlePositionX[totalIdxs[saIdx]]
            deltaX = SpatialGroups._delta_x(soredPosition, soredPosition[:, np.newaxis], 10, 5)
            distances = SpatialGroups.distance_x(deltaX)
            Aij = distances <= sa.model.distanceD0
            colors = totalColors[saIdx]
            # for idx, row in enumerate(Aij):
            #     isTrue = np.where(row)[0]
            #     ax2.scatter(np.ones_like(isTrue) * idx, isTrue, color=colors[idx], s=2, alpha=0.6)
            for idx, row in enumerate(Aij):
                isTrue = np.where(row)[0]
                subColors = np.array(colors[idx])[isTrue]
                # print(subColors)
                # print(type(subColors))
                # print(subColors)
                ax2.scatter(np.ones_like(isTrue) * idx, isTrue, color=subColors, s=1, alpha=0.6)

            ax2.tick_params(direction='in', length=3)
            ax2.spines['right'].set_color('none')
            ax2.spines['top'].set_color('none')
            ax2.set_xlabel(r"$i$", fontsize=16, labelpad=-10)
            ax2.set_ylabel(r"$j$", fontsize=16, rotation=0, labelpad=-10)
            ax2.grid(True)
            ax2.set_xlim(-0.5, len(totalIdxs[saIdx]) + 0.5)
            ax2.set_ylim(-0.5, len(totalIdxs[saIdx]) + 0.5)

        # plt.tight_layout()

    if earlyStop is None:
        TNum = sas[0].TNum
    else:
        TNum = min(earlyStop, sas[0].TNum)

    totalClasses = list()
    totalColors = list()
    totalIdxs = list()

    for saIdx in range(len(sas)):
        sa = sas[saIdx]
        lastPositionX, lastPhaseTheta, lastPointTheta = sa.get_state(index=TNum - 1)
        centers = StateAnalysis._calc_centers(
            lastPositionX, lastPhaseTheta, lastPointTheta, 3, 0.01
        )
        classes = StateAnalysis._calc_classes(
            centers, 0.3, StateAnalysis._adj_distance(centers, centers[:, np.newaxis], 10, 5)
        )
        drifters = list()
        ajdClasses = dict()
        classCounts = 0

        for classIdx in range(len(classes)):
            if len(classes[classIdx]) < 10:
                drifters.extend(classes[classIdx])
            else:
                ajdClasses[classCounts] = classes[classIdx]
                classCounts += 1
        clusterColorsMap = {str(i): cmap(i / len(ajdClasses) * 0.75 + 0.25) for i in range(len(ajdClasses))}
        clusterColorsKeys = list(clusterColorsMap.keys())
        adjClassColors = [
            [clusterColorsKeys[i]] * len(ajdClasses[i]) 
            for i in ajdClasses
        ] + [["gray"] * len(drifters)]

        colors = list()
        for i in range(len(adjClassColors)):
            colors.extend(adjClassColors[i])
        colors = np.array([colors] * sa.model.agentsNum).T
        lowerRight = np.tril(colors)
        for i in range(sa.model.agentsNum):
            for j in range(i+1, sa.model.agentsNum):
                colors[i, j] = lowerRight[j, i]
        
        # print(colors.shape)
        colors = colors.T.tolist()
        # print(len(colors[0]))
        for idx, row in enumerate(colors):
            subColors = colors[idx]
            # print(subColors)
            for subCIdx, subC in enumerate(subColors):
                if subC in clusterColorsMap:
                    colors[idx][subCIdx] = clusterColorsMap[subC]

        idxs = np.concatenate(list(ajdClasses.values()) + [drifters]).astype(np.int32)
        
        totalClasses.append(ajdClasses)
        totalColors.append(colors)
        totalIdxs.append(idxs)

    fig, axs = plt.subplots(2, 3, figsize=(10, 2 * 3))
    axs = axs.flatten()
    
    startIdx = TNum - tailCounts * step if tailCounts else 0
    frames = np.arange(startIdx, TNum, step)
    pbar = tqdm(total=len(frames))
    ani = ma.FuncAnimation(fig, plot_frame, frames=frames, interval=20 * step, repeat=False)
    ani.save(f"{mp4Path}/{fileName}.mp4", dpi=200)
    plt.close()
    pbar.close()

In [41]:
models = [
    SpatialGroups(strengthLambda=0.015, distanceD0=0.25, randomSeed=10, savePath="./data"),
    SpatialGroups(strengthLambda=0.015, distanceD0=0.35, randomSeed=10, savePath="./data"),
    SpatialGroups(strengthLambda=0.015, distanceD0=0.9, randomSeed=10, savePath="./data")
]
sas = [StateAnalysis(model) for model in models]
draw_mp4s(sas, "V#8~10_chiral_CS", tailCounts=750, step=1)

  0%|          | 0/750 [00:00<?, ?it/s]

In [42]:
models = [
    SpatialGroups(strengthLambda=0.1, distanceD0=0.45, randomSeed=10, savePath="./data"),
    SpatialGroups(strengthLambda=0.1, distanceD0=0.75, randomSeed=10, savePath="./data"),
    SpatialGroups(strengthLambda=0.1, distanceD0=1.5, randomSeed=10, savePath="./data")
]
sas = [StateAnalysis(model) for model in models]
draw_mp4s(sas, "V#11~13_chiral_CLS", tailCounts=750, step=1)

  0%|          | 0/750 [00:00<?, ?it/s]

In [44]:
models = [
    SingleDistribution(strengthLambda=0.01, distanceD0=0.35, randomSeed=10, savePath="./data"),
    SingleDistribution(strengthLambda=0.01, distanceD0=0.65, randomSeed=10, savePath="./data"),
    SingleDistribution(strengthLambda=0.01, distanceD0=1.05, randomSeed=10, savePath="./data")
]
sas = [StateAnalysis(model) for model in models]
draw_mp4s(sas, "V#2~4_monoChiral_CS", tailCounts=750, step=1)

  0%|          | 0/750 [00:00<?, ?it/s]

In [3]:
models = [
    SingleDistribution(strengthLambda=0.01, distanceD0=0.35, randomSeed=10, savePath="./data"),
    SingleDistribution(strengthLambda=0.01, distanceD0=0.65, randomSeed=10, savePath="./data"),
    SingleDistribution(strengthLambda=0.01, distanceD0=1.05, randomSeed=10, savePath="./data")
]
sas = [StateAnalysis(model) for model in models]
draw_mp4s(sas, "V#2~4_monoChiral_CS_ap", tailCounts=750, step=1)

  0%|          | 0/750 [00:00<?, ?it/s]

In [43]:
models = [
    SingleDistribution(strengthLambda=0.15, distanceD0=0.5, randomSeed=10, savePath="./data"),
    SingleDistribution(strengthLambda=0.15, distanceD0=0.85, randomSeed=10, savePath="./data"),
    SingleDistribution(strengthLambda=0.15, distanceD0=0.95, randomSeed=10, savePath="./data")
]
sas = [StateAnalysis(model) for model in models]
draw_mp4s(sas, "V#5~7_monoChiral_CLS", tailCounts=750, step=1)

  0%|          | 0/750 [00:00<?, ?it/s]

In [10]:
model = SingleDistribution(strengthLambda=0, distanceD0=0, randomSeed=10, savePath="./data", overWrite=True, tqdm=True, boundaryLength=10)
sa = StateAnalysis(model)
draw_mp4(sa, "V#1_monoChiral_lambda0", tailCounts=750, step=1)

  0%|          | 0/750 [00:00<?, ?it/s]

In [14]:
model = SpatialGroups(strengthLambda=0.02, distanceD0=0.4, randomSeed=80, savePath="./data")
sa = StateAnalysis(model)
draw_time_vary(sa, step=1, earlyStop=5000)

Calculating order parameters:   0%|          | 0/4999 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [7]:
models = [
    SingleDistribution(strengthLambda=0.01, distanceD0=0.35, randomSeed=10, savePath="./data"),
    SingleDistribution(strengthLambda=0.01, distanceD0=0.65, randomSeed=10, savePath="./data"),
    SingleDistribution(strengthLambda=0.01, distanceD0=1.05, randomSeed=10, savePath="./data")
]
sas = [StateAnalysis(model) for model in models]
draw_mp4s_row2_Aij(sas, "V#2~4_monoChiral_CS_Aij", tailCounts=750, step=1)

  0%|          | 0/750 [00:00<?, ?it/s]

d:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:324: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [8]:
models = [
    SingleDistribution(strengthLambda=0.15, distanceD0=0.5, randomSeed=10, savePath="./data"),
    SingleDistribution(strengthLambda=0.15, distanceD0=0.85, randomSeed=10, savePath="./data"),
    SingleDistribution(strengthLambda=0.15, distanceD0=0.95, randomSeed=10, savePath="./data")
]
sas = [StateAnalysis(model) for model in models]
draw_mp4s_row2_Aij(sas, "V#5~7_monoChiral_CLS_Aij", tailCounts=750, step=1)

  0%|          | 0/750 [00:00<?, ?it/s]

d:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:324: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [2]:
import cv2
import os

# 设置输入文件夹路径
input_folder = './normalMP4/SingleDistribution/'

# 设置输出视频文件名和格式
output_file = './normalMP4/SingleDistribution.mp4'

# 获取文件夹中的所有 PNG 图片文件
image_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.png')])

# 获取第一张图片的尺寸
first_image = cv2.imread(os.path.join(input_folder, image_files[0]))
height, width, _ = first_image.shape

# 创建视频写入对象
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_file, fourcc, 50, (width, height))

# 逐帧读取图片并写入视频
for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    image = cv2.imread(image_path)
    video_writer.write(image)

# 释放视频写入对象
video_writer.release()

print(f"Video saved as {output_file}")

Video saved as ./normalMP4/SingleDistribution.mp4
